# Data analysis for MCMC calculations of energies
## Problem 1c) Importance sampling
### Fabian Faulstich and Yngve Mardal Moe

In [1]:
# Base python imports
import numpy as np
import matplotlib.pyplot as plt
import joblib
from time import time
from timeit import timeit

# VMC imports
from particles import Particles
from wave_function import WaveFunction
from sampler import MetropolisSampler, ImportanceSampler

In [2]:
# Supress divide by zero warnings
np.seterr(divide='ignore')

# IPython magics
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Problem 1 b):
We start by defining a simple function that simulate the system according to the parameters given in problem 1b)

In [3]:
def create_sampler(num_particles, num_dimensions, alpha, timestep, seed=None):
    parts = Particles(
        num_particles=num_particles, 
        num_dimensions=num_dimensions,
        mass=1,
        diameter=0.0043
    )
    wave_function = WaveFunction(
        particles=parts,
        alpha=alpha,
        beta=1,
        omega_ho=1,
        omega_z=1
    )
    sampler = ImportanceSampler(
        wave_function=wave_function,
        timestep=timestep,
        seed=seed
    )
    return sampler


def simulate_simple(num_particles, num_dimensions, alpha, timestep,
                    num_iterations, seed=None):
    sampler = create_sampler(
        num_particles=num_particles,
        num_dimensions=num_dimensions,
        alpha=alpha,
        timestep=timestep,
        seed=seed
    )
    return sampler.compute_local_energy(num_iterations)

We now find good step sizes for our samplers.

In [4]:
alphas = np.concatenate((np.logspace(-2, np.log(.5)/np.log(10), 3, base=10),np.logspace(0, 2, 2, base=10)))
num_particles = np.array([1, 10])#, 100, 500])
num_dimensions = np.array([3]) # 1, 2,
samplers = {}


In [5]:
for alpha in alphas:
    for n in num_particles:
        for d in num_dimensions:
            samplers[(alpha, n, d)] = create_sampler(n, d, alpha, 0.1)

In [6]:
acceptance_ratios = {}
for params, sampler in samplers.items():
    print(f'Params: {params}')
    print('-'*50)
    t0 = time()
    try:
        acceptance_ratios[params] = sampler.find_ideal_step_size(
            delta=0.01,
            ideal=0.85,
            num_iterations=1000,
            verbose=True
        )
    except RuntimeError:
        print('Failed to find ideal step size, we try again with a larger'
              'number of iterations.')
        print('-'*50)
        acceptance_ratios[params] = sampler.find_ideal_step_size(
            delta=0.01,
            ideal=0.85,
            num_iterations=2000,
            verbose=True
        )
    print(f'Took {time()-t0:3f} seconds')
    print('='*50)

for params, ratios in acceptance_ratios.items():
    print(f'{params}: step_size: {ratios[0]:03f}, acceptance_ratio: {ratios[1]:03f}')

Params: (0.01, 1, 3)
--------------------------------------------------


/home/yngve/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yngve/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Found left bound, 0.1
  With ar 1.0
Found right bound, 51.2
  With ar 0.816

Starting iterations now
    Iteration number 1 completed
Found acceptable step size after 1 iterations
Took 7.593598 seconds
Params: (0.01, 10, 3)
--------------------------------------------------
Found left bound, 0.1
  With ar 1.0
Found right bound, 51.2
  With ar 0.828

Starting iterations now
    Iteration number 1 completed
    Iteration number 2 completed
    Iteration number 3 completed
    Iteration number 4 completed
    Iteration number 5 completed
    Iteration number 6 completed
    Iteration number 7 completed
Found acceptable step size after 7 iterations
Took 10.930489 seconds
Params: (0.070710678118654752, 1, 3)
--------------------------------------------------
Found left bound, 0.1
  With ar 0.999
Found right bound, 12.8
  With ar 0.629

Starting iterations now
    Iteration number 1 completed
    Iteration number 2 completed
Found acceptable step size after 2 iterations
Took 6.362630 seconds

/home/yngve/Dropbox/Skole/UiO/FYS4411/FYS4411-VMC/python/sampler.py:210: RuntimeWarning: invalid value encountered in longdouble_scalars
  return self.greens_ratio*self.P/(self._old_P)


Found left bound, 0.00078125
  With ar 0.933
Found right bound, 0.1
  With ar 0.0

Starting iterations now
    Iteration number 1 completed
    Iteration number 2 completed
    Iteration number 3 completed
    Iteration number 4 completed
    Iteration number 5 completed
    Iteration number 6 completed
Found acceptable step size after 6 iterations
Took 6.215864 seconds
(0.01, 1, 3): step_size: 25.650000, acceptance_ratio: 0.856000
(0.01, 10, 3): step_size: 23.653906, acceptance_ratio: 0.852000
(0.070710678118654752, 1, 3): step_size: 3.275000, acceptance_ratio: 0.844000
(0.070710678118654752, 10, 3): step_size: 3.479639, acceptance_ratio: 0.854000
(0.50000000000000011, 1, 3): step_size: 0.475000, acceptance_ratio: 0.849000
(0.50000000000000011, 10, 3): step_size: 0.475000, acceptance_ratio: 0.852000
(1.0, 1, 3): step_size: 0.275000, acceptance_ratio: 0.845000
(1.0, 10, 3): step_size: 0.231250, acceptance_ratio: 0.858000
(100.0, 1, 3): step_size: 0.002332, acceptance_ratio: 0.857000
(1

-------
With step sizes found, we compute the energies and compare them to the true energies.

The local energy of a harmonic oscillator (which the non-interacting case is) is given by
$$nd\alpha + ||\vec{r}||^2\left[\frac{1}{2} - 2 \alpha^2\right],$$
where $n$ is the number of particles and $d$ is the number of dimensions of the system.

We then compute the expected value $\mathbb{E}[E_L]$ using $\hbar = m = \omega_{ho} = \omega_\ = 1$ and get
$$\mathbb{E}[E_L] = \frac{nd}{4\alpha}$$

Let us now check if these match the simulated parameters.

Before we do that, we "warm up" the samplers so that the particles have reasonable initial conditions and time this to get an estimate of how long time the MCMC iterations will take.

In [7]:
acceptance_ratios = {}
for param, sampler in samplers.items():
    print(param)
    acceptance_ratios[param] = sampler.compute_acceptance_ratio(1000)

(0.01, 1, 3)
(0.01, 10, 3)
(0.070710678118654752, 1, 3)
(0.070710678118654752, 10, 3)
(0.50000000000000011, 1, 3)
(0.50000000000000011, 10, 3)
(1.0, 1, 3)
(1.0, 10, 3)
(100.0, 1, 3)
(100.0, 10, 3)


In [8]:
energies = {}
variances = {}
for param, sampler in samplers.items():
    print(param)
    e, v = sampler.compute_local_energy(1000)
    energies[param] = e
    variances[param] = v

(0.01, 1, 3)
(0.01, 10, 3)
(0.070710678118654752, 1, 3)
(0.070710678118654752, 10, 3)
(0.50000000000000011, 1, 3)
(0.50000000000000011, 10, 3)
(1.0, 1, 3)
(1.0, 10, 3)
(100.0, 1, 3)
(100.0, 10, 3)


In [13]:
def expected_energy(parameters):
    return np.prod(parameters)/2 + parameters[1]*parameters[2]/(parameters[0]*8)

acceptance_ratios = {param: sampler.acceptance_ratio for param, sampler in samplers.items()}

print('| Alpha |  n  | d | Computed energy | True energy |   Error   |   Variances  | Acceptance rate |')
print('|-------|-----|---|-----------------|-------------|-----------|--------------|-----------------|')
for param, energy in energies.items():
    print(f'|{param[0]:6.3f} |{param[1]:4d} | {param[2]} |{energy:16.4f} |{expected_energy(param):12.3f} |{energy - expected_energy(param):10.3f} | {variances[param]:12.1f} |  {acceptance_ratios[param]:14.4f} |')

| Alpha |  n  | d | Computed energy | True energy |   Error   |   Variances  | Acceptance rate |
|-------|-----|---|-----------------|-------------|-----------|--------------|-----------------|
0.07759403154
0.00577112899812
0.0151461041394
-0.0230484379473
0.0
0.0
-0.0209147816201
-0.0347081064092
0.0287590350476
0.018437592307


This table comes from copying the above output

| Alpha |  n  | d | Computed energy | True energy |   Error   |   Variances  | Acceptance rate |
|-------|-----|---|-----------------|-------------|-----------|--------------|-----------------|
| 0.010 |   1 | 1 |          7.7851 |      12.505 |    -4.720 |        132.4 |          0.8025 |
| 0.010 |   1 | 2 |         11.8874 |      25.010 |   -13.123 |        165.6 |          0.8165 |
| 0.010 |   1 | 3 |         23.3768 |      37.515 |   -14.138 |        346.8 |          0.7975 |
| 0.010 |  10 | 1 |         63.6476 |     125.050 |   -61.402 |        909.7 |          0.7885 |
| 0.010 |  10 | 2 |        124.3144 |     250.100 |  -125.786 |       2203.1 |          0.7930 |
| 0.010 |  10 | 3 |        168.0565 |     375.150 |  -207.093 |       2300.4 |          0.8040 |
| 0.027 |   1 | 1 |          2.5254 |       4.714 |    -2.189 |         12.3 |          0.8130 |
| 0.027 |   1 | 2 |          4.0483 |       9.428 |    -5.380 |         14.1 |          0.7685 |
| 0.027 |   1 | 3 |          6.7754 |      14.142 |    -7.367 |         34.7 |          0.7925 |
| 0.027 |  10 | 1 |         22.3944 |      47.140 |   -24.746 |         91.6 |          0.8065 |
| 0.027 |  10 | 2 |         52.9504 |      94.281 |   -41.331 |        246.0 |          0.7960 |
| 0.027 |  10 | 3 |         67.6719 |     141.421 |   -73.750 |        154.7 |          0.8105 |
| 0.071 |   1 | 1 |          0.9597 |       1.803 |    -0.843 |          1.4 |          0.7870 |
| 0.071 |   1 | 2 |          1.9669 |       3.606 |    -1.639 |          3.1 |          0.7900 |
| 0.071 |   1 | 3 |          2.9783 |       5.409 |    -2.431 |          5.4 |          0.8060 |
| 0.071 |  10 | 1 |         10.3897 |      18.031 |    -7.641 |         11.3 |          0.7770 |
| 0.071 |  10 | 2 |         21.1092 |      36.062 |   -14.953 |         52.3 |          0.7820 |
| 0.071 |  10 | 3 |         27.3985 |      54.094 |   -26.695 |         26.1 |          0.8125 |
| 0.188 |   1 | 1 |          0.4594 |       0.759 |    -0.299 |          0.1 |          0.7885 |
| 0.188 |   1 | 2 |          1.0388 |       1.518 |    -0.479 |          0.7 |          0.8015 |
| 0.188 |   1 | 3 |          1.4108 |       2.276 |    -0.866 |          0.4 |          0.7945 |
| 0.188 |  10 | 1 |          4.3130 |       7.588 |    -3.275 |          1.1 |          0.7980 |
| 0.188 |  10 | 2 |          9.6284 |      15.176 |    -5.548 |          3.7 |          0.7900 |
| 0.188 |  10 | 3 |         13.7083 |      22.764 |    -9.056 |          2.6 |          0.8045 |
| 0.500 |   1 | 1 |          0.5000 |       0.500 |     0.000 |          0.0 |          0.8065 |
| 0.500 |   1 | 2 |          1.0000 |       1.000 |     0.000 |          0.0 |          0.8060 |
| 0.500 |   1 | 3 |          1.5000 |       1.500 |     0.000 |          0.0 |          0.7870 |
| 0.500 |  10 | 1 |          5.0000 |       5.000 |     0.000 |          0.0 |          0.7830 |
| 0.500 |  10 | 2 |         10.0000 |      10.000 |     0.000 |          0.0 |          0.8095 |
| 0.500 |  10 | 3 |         15.0000 |      15.000 |     0.000 |          0.0 |          0.7865 |

Thus, we see that our MCMC sampler does indeed seem to give the correct result for the non-interacting case.

----------------------
## Timings
Let us now see what the complexity of the MCMC sampler is. Here, $m$ denote the number of iterations, $n$ the number of particles and $d$ the number of dimensions.

We choose alpha equal to 0.5 for these tests.

The computational complexity scales linearly with the number of iterations, $m$, as each iteration does the exact same thing. We therefore only need to test the computational complexity wrt $n$ and $d$.

In [10]:
alpha = 0.5
num_particles = np.logspace(0, 2, 10, dtype=int)
num_dimensions = np.array([1, 2, 3])
speed_samplers = {}

for n in num_particles:
    for d in num_dimensions:
        speed_samplers[(n, d)] = create_sampler(n, d, alpha, 0.1)

In [11]:
for i, sampler in enumerate(speed_samplers.values()):
    print(i)
    sampler.find_ideal_step_size(0.01, 0.9, num_iterations=1000, verbose=True)

0


/home/yngve/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yngve/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Found left bound, 0.1
  With ar 0.993
Found right bound, 1.6
  With ar 0.842

Starting iterations now
    Iteration number 1 completed
    Iteration number 2 completed
    Iteration number 3 completed
    Iteration number 4 completed
Found acceptable step size after 4 iterations
1
Found left bound, 0.1
  With ar 0.988
Found right bound, 1.6
  With ar 0.752

Starting iterations now
    Iteration number 1 completed
    Iteration number 2 completed
    Iteration number 3 completed
    Iteration number 4 completed
    Iteration number 5 completed
Found acceptable step size after 5 iterations
2
Found left bound, 0.1
  With ar 0.986
Found right bound, 0.8
  With ar 0.889

Starting iterations now
    Iteration number 1 completed
    Iteration number 2 completed
    Iteration number 3 completed
Found acceptable step size after 3 iterations
3
Found left bound, 0.1
  With ar 0.992
Found right bound, 1.6
  With ar 0.849

Starting iterations now
    Iteration number 1 completed
    Iteration numbe

KeyboardInterrupt: 

In [ ]:
timings = {}
num_it = 100
for params, sampler in speed_samplers.items():
    print(params)
    timings[params] = timeit(lambda : sampler.compute_local_energy(num_it),
                             number=100)
    

In [ ]:
plt.figure()
for d in num_dimensions:
    runtime = [timings[(n, d)]/num_it for n in num_particles]
    plt.plot(num_particles, runtime)

plt.legend([f'{d} dimensions' for d in num_dimensions])
plt.title('Runtime per iteration')
plt.xlabel('Number of particles')
plt.ylabel('Time [s]')
plt.show()

This seem like it might be squared complexity. Let us therefore perform a square root transform.

In [ ]:
plt.figure()
for d in num_dimensions:
    runtime = np.array([timings[(n, d)]/num_it for n in num_particles])
    plt.plot(num_particles, np.sqrt(runtime))
    plt.title('Root runtime as a function of number of particles')

plt.legend([f'{d} dimensions' for d in num_dimensions])
plt.title('Runtime per iteration')
plt.xlabel('Number of particles')
plt.ylabel('Time [s]')
plt.show()

From this plot, it seems like there is a quadratic complexity with respect to the number of parameters (in the non-interacting case that is. In the interacting case there is a tripple loop which is skipped if the Boson diameter is 0).

# Let us now see if the position vector follows the correct distribution.